# Setup

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import time
import datetime
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import json
import pickle
np.random.seed(42)

from formats import experiment_pb2
from formats import  quantification_pb2

from skimage import io
import pandas as pd
import utils


from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset, ChainDataset


import os
import pyro
import pyro.distributions as dist
import pyro.poutine
from pyro.infer import MCMC, NUTS
import math
import torch.nn as nn
import torch.nn.functional as F
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer.autoguide.guides import AutoDiagonalNormal
import pyro.distributions.constraints as constraints
from tqdm import trange

import utils
import umap


/home/austin/.conda/envs/JPLEnv/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/austin/.conda/envs/JPLEnv/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/austin/.conda/envs/JPLEnv/lib/python3.9/site-packages/umap/distances.py:

In [2]:
local_radius_px = 5

In [3]:
import data
dataset = data.get_dataset(local_radius_px=local_radius_px)

In [28]:
subsample = 0.1
x,q = next(iter(torch.utils.data.DataLoader(dataset=dataset,batch_size=int(subsample*len(dataset)))))
x /= x.shape[1]*x.shape[2]/q.shape[1]


In [29]:
def eval_reconstruction(reconstruction):
    x_hat, q_hat = reconstruction
    
    x_flat = x.reshape(-1,x.shape[-1])
    x_hat = x_hat.reshape(-1,x.shape[-1])

    x_mean = x_flat.mean(0)
    x_ssr = (x_flat - x_hat).pow(2).sum()
    x_sst = (x_flat - x_mean).pow(2).sum()
    x_r2 = 1 - x_ssr/x_sst

    q_mean = q.mean(0)
    q_ssr = (q - q_hat).pow(2).sum()
    q_sst = (q - q_mean).pow(2).sum()
    q_r2 = 1 - q_ssr/q_sst

    return x_r2.item(), q_r2.item()

In [30]:
def train_joint_umap(n_components):
    combined_data = torch.cat((q,x.reshape((x.shape[0],-1))),1)

    umap_model = umap.UMAP(n_components=n_components,random_state=42)
    umap_model = umap_model.fit(combined_data) 
    return umap_model
    

def joint_umap_reconstruct(umap_model):
    combined_data = torch.cat((q,x.reshape((x.shape[0],-1))),1)
    umap_enc = umap_model.transform(combined_data) 
    recon = torch.tensor(umap_model.inverse_transform(umap_enc))
    q_hat = recon[:,:q.shape[1]] 
    x_hat = recon[:,q.shape[1]:].reshape(x.shape) 
    return x_hat, q_hat

def joint_umap_latents(umap_model):
    combined_data = torch.cat((q,x.reshape((x.shape[0],-1))),1)
    umap_enc = umap_model.transform(combined_data) 
    return umap_enc

def save_umap_file(n_components):
    print('training...',end='\r')
    umap_model = train_joint_umap(n_components=n_components)
    umap_latents = joint_umap_latents(umap_model)
    print('reconstructing...',end='\r')
    umap_recon = joint_umap_reconstruct(umap_model)
    umap_dict = {
        'z' : umap_latents,
        'x_hat' : umap_recon[0].numpy(),
        'q_hat' : umap_recon[1].numpy(),
        'x':x.numpy(),
        'q':q.numpy()
    }
    dbfile = open(f'models/umap-z-{n_components}.pkl', 'ab')
    pickle.dump(umap_dict,dbfile)
    dbfile.close()
    print(eval_reconstruction(umap_recon))


In [31]:
save_umap_file(2)

(0.680641770362854, 0.6511508226394653)


In [32]:
save_umap_file(3)

(0.695252537727356, 0.6331181526184082)
